In [2]:
import mediapipe as mp
import numpy as np
import cv2 as cv
import os
# import tensorflow as tf
# from model.keypoint_classifier.coral_keypoint_classifier import CoralKeyPointClassifier
# from model import PointHistoryClassifier
from model.keypoint_classifier.keypoint_classifier import KeyPointClassifier
from utils import CvFpsCalc
import copy
import itertools
from collections import Counter
from collections import deque
print(os.getcwd())

import csv

model_path='model/keypoint_classifier/keypoint_classifier.tflite'


2022-12-14 22:48:50.250562: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 22:48:50.422359: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-14 22:48:51.029028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pumid/Desktop/athome/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.2/lib64:/home/pumid/ros2_ws/install/realsense2_camera/lib:/home/pumid/ros2_ws/install/realsense2_cam

This module is in /home/pumid/Desktop/athome/athome_ws/src/gesture_detection/scripts/model/keypoint_classifier
Your are using Tensorflow library
/home/pumid/Desktop/athome/athome_ws/src/gesture_detection/scripts


# Classifier Function

In [3]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv.boundingRect(landmark_array)

    return [x, y, x + w, y + h]

# History Function

In [4]:
def pre_process_point_history(image, point_history):
    image_width, image_height = image.shape[1], image.shape[0]

    temp_point_history = copy.deepcopy(point_history)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, point in enumerate(temp_point_history):
        if index == 0:
            base_x, base_y = point[0], point[1]

        temp_point_history[index][0] = (temp_point_history[index][0] - base_x) / image_width
        temp_point_history[index][1] = (temp_point_history[index][1] - base_y) / image_height

    # Convert to a one-dimensional list
    temp_point_history = list(
        itertools.chain.from_iterable(temp_point_history))

    return temp_point_history
    
def draw_point_history(image, point_history):
    for index, point in enumerate(point_history):
        if point[0] != 0 and point[1] != 0:
            cv.circle(image, (point[0], point[1]), 1 + int(index / 2),
                      (152, 251, 152), 2)

    return image

In [28]:
##### argument #####
cap_device = 0
cap_width = 960
cap_height = 540
use_static_image_mode = False
min_detection_confidence = 0.7
min_tracking_confidence = 0.5

##### setting camera #####
cap = cv.VideoCapture()
cap.set(cv.CAP_PROP_FRAME_WIDTH, cap_width)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, cap_height)

##### mediapipe hand #####
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
        static_image_mode=use_static_image_mode,
        max_num_hands=1,
        min_detection_confidence=min_detection_confidence,
        min_tracking_confidence=min_tracking_confidence,
    )

##### load model #####
keypoint_classifier = KeyPointClassifier()
keypoint_classifier_labels = ['Open', 'start_command',  'Close', 'Pointer', 'Open']
# point_history_classifier = PointHistoryClassifier()
# point_history_classifier_labels = ['Stop', 'Clockwise', 'Counter Clockwise', 'Move']


##### measure FPS #####
cvFpsCalc = CvFpsCalc(buffer_len=10)

##### declare history point
history_length = 16
point_history = deque(maxlen=history_length)
    
# Finger gesture history ################################################
finger_gesture_history = deque(maxlen=history_length)

##### measure fps
fps_que = deque(maxlen=50)

x = [260, 220, 180, 150, 135, 120, 100, 80, 65, 55, 45, 40, 38, 35, 30, 29, 25, 21, 85, 73, 60, 52, 48, 42, 36, 34, 31, 28, 26, 25, 23, 22, 21, 20, 19, 18, 17, 16, 16, 16, 15, 15, 140, 185, 105]
y = [5, 10, 15, 20 ,25, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 20, 15, 30]


fit_distance = np.polyfit(x, y, 2)
p = np.poly1d(fit_distance)


In [17]:
fit_distance[0]*(distance**2)+fit_distance[1]*(distance)+fit_distance[2]

20.914655386048963

# Video Loop

In [29]:
##### setting camera #####
cap = cv.VideoCapture(cap_device)
cap.set(cv.CAP_PROP_FRAME_WIDTH, cap_width)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, cap_height)
while cap.isOpened():
    fps = cvFpsCalc.get()
    fps_que.append(fps)

    if cv.waitKey(10) & 0xFF == ord('q'):
        break

    ret, image = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    image = cv.flip(image, 1)
    debug_image = copy.deepcopy(image)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True

    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            ##### test normalize 
            landmark_point = []
            for _, landmark in enumerate(hand_landmarks.landmark):
                landmark_point.append([landmark.x, landmark.y, landmark.z])
            # print(landmark.z)
            ##### mediapipe output normarlize convert to pixel
            landmark_list = calc_landmark_list(debug_image, hand_landmarks)
            x1, y1 = landmark_list[5]

            x2, y2 = landmark_list[17]

            distance = int(np.sqrt((y2-y1)**2 + (x2-x1)**2))
            # print(p(distance))
            world_distance = fit_distance[0]*(distance**2)+fit_distance[1]*(distance)+fit_distance[2]

            ##### hand classifier
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            # print(len(pre_processed_landmark_list))

            hand_sign_id = keypoint_classifier(pre_processed_landmark_list)

            # ##### store hand classifier
            # if hand_sign_id == 2:  # Point gesture
            #     point_history.append(landmark_list[8])
            # else:
            #     point_history.append([0, 0])

            # ##### point history classifier
            # finger_gesture_id = 0
            # if len(point_history) == history_length:
            #     pre_processed_point_history_list = pre_process_point_history(debug_image, point_history)
            #     finger_gesture_id = point_history_classifier(pre_processed_point_history_list)

            # # Calculates the gesture IDs in the latest detection
            # finger_gesture_history.append(finger_gesture_id)
            # most_common_fg_id = Counter(finger_gesture_history).most_common()
            # dynamic_sign = point_history_classifier_labels[most_common_fg_id[0][0]]


            ##### display information|
            ### Draw Landmarks
            mp_drawing.draw_landmarks(debug_image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())

            ### draw bounding box 
            brect = calc_bounding_rect(debug_image, hand_landmarks)
            cv.rectangle(debug_image, (brect[0], brect[1]), (brect[2], brect[3]), (0, 0, 0), 1)

            ### draw info text
            cv.rectangle(debug_image, (brect[0], brect[1]), (brect[2], brect[1] - 22),(0, 0, 0), -1)
            hand_sign_text = keypoint_classifier_labels[hand_sign_id]
            info_text = handedness.classification[0].label[0:]
            if hand_sign_text != "":
                info_text = info_text + ':' + hand_sign_text
            cv.putText(debug_image, info_text, (brect[0] + 5, brect[1] - 4), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv.LINE_AA)
            cv.putText(debug_image, "Distance:" + str(np.round(distance, 2)) + ' cm', (10, 100), cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv.LINE_AA)
            cv.putText(debug_image, "Distance:" + str(np.round(distance, 2)) + ' cm', (10, 100), cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv.LINE_AA)
            # finger_gesture_text = dynamic_sign
            # if finger_gesture_text != "":
            #     cv.putText(debug_image, "Finger Gesture:" + finger_gesture_text, (10, 60), cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv.LINE_AA)
            #     cv.putText(debug_image, "Finger Gesture:" + finger_gesture_text, (10, 60), cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv.LINE_AA)

                
        
    debug_image = draw_point_history(debug_image, point_history)
    cv.putText(debug_image, "FPS:" + str(fps), (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv.LINE_AA)
    cv.putText(debug_image, "FPS:" + str(fps), (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv.LINE_AA)

    cv.imshow('Raw Webcam Feed', debug_image)        

cap.release()
cv.destroyAllWindows()

14.800000000000018

# Logging Csv

In [ ]:
csv_path = 'model/keypoint_classifier/test_keypoint.csv'
##### setting camera #####
cap = cv.VideoCapture(cap_device)
cap.set(cv.CAP_PROP_FRAME_WIDTH, cap_width)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, cap_height)
while cap.isOpened():
    fps = cvFpsCalc.get()

    if cv.waitKey(10) & 0xFF == ord('q'):
        break

    ret, image = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    image = cv.flip(image, 1)
    debug_image = copy.deepcopy(image)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True

    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            ##### mediapipe output normarlize convert to pixel
            landmark_list = calc_landmark_list(debug_image, hand_landmarks)

            ##### hand classifier
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            with open(csv_path, 'a', newline="") as f:
                writer = csv.writer(f)
                writer.writerow([1, *pre_processed_landmark_list])

            ##### display information|
            ### Draw Landmarks
            mp_drawing.draw_landmarks(debug_image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())

    cv.imshow('Raw Webcam Feed', debug_image)        

cap.release()
cv.destroyAllWindows()

In [ ]:
from collections import Counter
c = Counter([1, 1, 2, 3, 1, 1, 1, 0])
print(c)